# Understanding the effects of PyCIGAR hyperparameters on training and results

The goal of this notebook is to gain intuition on the impact of the different hyperparameters currently specified to train a PPO agent in PyCIGAR.

Fixed settings in this experiment:
- PPO algorithm
- `ieee37busdata network` and load profile
- tracked device: `inverter_s701a`
- discrete single actions
- `CentralControlPVInverterEnv` environment


Hyperparameters identified:
- discount factor $\gamma$
- GAE lambda
- train batch size
- depth of NN
- widths of NN layers
- lr (or lr schedule)
- loss factors (penalties)
    - oscillation
    - action change
    - deviation from the initial command
    
    

Until now, mostly qualitative results were obtained, mostly by looking at the graphs of voltage, $y$ value, injected power and actions taken by a tracked device over the course of a test simulation. In order to have objective grounds on which we can compare different solutions, we need quantitative results. Some statistics that reflect different aspects of both the solutions and the training processes are listed in the next section.

## Statistics that summarize a training

foreach epoch: 
    - number of actions taken
    - average magnitude of the actions
    - total reward
    - time of earliest action
    - average Shannon entropy of the action distribution

- epoch at which the policy does not change anymore
- average runtime of an epoch


An additional qualitative result could be a GIF of the curves over epochs


## Methods to understand hyperparameters

Some methods that come to mind to obtain an intuition are:

- try extreme values, then compare statistics
- change independently: plot statistics
- bayesian optimization with statistics for objective


In [1]:
import os
import ray
from copy import deepcopy
os.chdir("/home/alex/ceds-cigar/rl/notebooks")
from understand_hyperparameters import base_config, get_train_fn, env_name, test_env

ray.init()

2020-03-02 11:50:01,583	INFO resource_spec.py:212 -- Starting Ray with 27.98 GiB memory available for workers and up to 13.99 GiB for objects. You can adjust these settings with ray.init(memory=<bytes>, object_store_memory=<bytes>).
2020-03-02 11:50:02,119	INFO services.py:1078 -- View the Ray dashboard at localhost:8265


{'node_ip_address': '128.3.28.231',
 'redis_address': '128.3.28.231:23941',
 'object_store_address': '/tmp/ray/session_2020-03-02_11-50-01_560675_5436/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2020-03-02_11-50-01_560675_5436/sockets/raylet',
 'webui_url': 'localhost:8265',
 'session_dir': '/tmp/ray/session_2020-03-02_11-50-01_560675_5436'}

In [2]:
N_WORKERS = 8
EPOCHS = 20
EVAL_ROUNDS = 3

In [3]:
base_config['num_workers'] = N_WORKERS
full_config = {
    'model_config': base_config,
    'pycigar_params': pycigar_params,
    'epochs': EPOCHS,
    'eval_rounds': EVAL_ROUNDS
}

## Gamma

Values for gamma are typically set to 0.95 or 0.99 in the litterature. However low values of gamma may be beneficial to some scenarios.

A value of 0 indicates greedy short term vision: the choice of an action is determined by the next reward.

A value of 1 indicates undiscounted rewards, where rewards of all future timesteps matter equally.

In [4]:
config = deepcopy(base_config)
config['gamma'] = ray.tune.grid_search([0, 0.3, 0.5, 0.9, 1])
    run_hp_experiment({'model_config': {'gamma': ray.tune.grid_search([0, 0.3, 0.5, 0.9, 1])}},
                      full_config, 'gamma')

Trial name,status,loc,gamma
coop_train_fn_01ce5c76,RUNNING,,
coop_train_fn_01cf1a8a,PENDING,,
coop_train_fn_01cf9492,PENDING,,
coop_train_fn_01d00e2c,PENDING,,


(pid=5503) 2020-03-02 11:50:05,655	INFO trainer.py:420 -- Tip: set 'eager': true or the --eager flag to enable TensorFlow eager execution
(pid=5503) 2020-03-02 11:50:05,686	INFO trainer.py:580 -- Current log_level is ERROR. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
(pid=5503) /home/alex/anaconda3/lib/python3.7/site-packages/ray/rllib/utils/from_config.py:134: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
(pid=5503)   obj = yaml.load(type_)
(pid=5503) 2020-03-02 11:50:09,569	WARNING util.py:37 -- Install gputil for GPU system monitoring.
  0%|          | 0/20 [00:00<?, ?it/s]
(pid=5496) /home/alex/anaconda3/lib/python3.7/site-packages/ray/rllib/utils/from_config.py:134: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
(

Result for coop_train_fn_01ce5c76:
  avg_entropy: .nan
  custom_metrics: {}
  date: 2020-03-02_11-52-36
  done: false
  earliest_action: 1.0
  episode_len_mean: 22.2
  episode_reward_max: -26.14741461764519
  episode_reward_mean: -35.200422563277066
  episode_reward_min: -50.962902008956064
  episodes_this_iter: 20
  episodes_total: 20
  experiment_id: 55afac1d235b4e1d9556c9e7ff43c5e7
  experiment_tag: 0_gamma=0
  hostname: gigteam
  info:
    grad_time_ms: 497.542
    learner:
      default_policy:
        cur_kl_coeff: 0.20000000298023224
        cur_lr: 0.00019999999494757503
        entropy: 1.5910038948059082
        entropy_coeff: 0.0
        kl: 0.01858106255531311
        policy_loss: -0.0375814214348793
        total_loss: 1.9940201044082642
        vf_explained_var: 0.0149829788133502
        vf_loss: 2.0278854370117188
    load_time_ms: 70.068
    num_steps_sampled: 500
    num_steps_trained: 384
    sample_time_ms: 87229.177
    update_time_ms: 773.479
  iterations_since_re

Trial name,status,loc,gamma,reward,total time (s),ts,iter
coop_train_fn_01ce5c76,RUNNING,128.3.28.231:5503,0,-35.2004,88.6483,500,1
coop_train_fn_01cf1a8a,PENDING,,,,,,
coop_train_fn_01cf9492,PENDING,,,,,,
coop_train_fn_01d00e2c,PENDING,,,,,,


(pid=5503) Running 3 evaluation rounds


...

Result for coop_train_fn_01d00e2c:
  avg_entropy: .nan
  custom_metrics: {}
  date: 2020-03-02_14-54-14
  done: false
  earliest_action: 0.0
  episode_len_mean: 22.24
  episode_reward_max: -14.002397741415791
  episode_reward_mean: -17.18398942825228
  episode_reward_min: -24.048503331328586
  episodes_this_iter: 25
  episodes_total: 445
  experiment_id: b6d1371fe6874c05951b2b75058d8cef
  experiment_tag: 3_gamma=0.9
  hostname: gigteam
  info:
    grad_time_ms: 195.074
    learner:
      default_policy:
        cur_kl_coeff: 0.010546875186264515
        cur_lr: 0.00019999999494757503
        entropy: 0.11284220963716507
        entropy_coeff: 0.0
        kl: 0.00027872747159563005
        policy_loss: -0.007401251699775457
        total_loss: 2.920513868331909
        vf_explained_var: -5.6544940889580175e-05
        vf_loss: 2.9279119968414307
    load_time_ms: 1.717
    num_steps_sampled: 10000
    num_steps_trained: 7680
    sample_time_ms: 79670.836
    update_time_ms: 7.42
  itera

Trial name,status,loc,gamma,reward,total time (s),ts,iter
coop_train_fn_01ce5c76,TERMINATED,,0,-33.3417,1616.1,10000,20
coop_train_fn_01cf1a8a,TERMINATED,,0.3,-19.0377,1623.6,10000,20
coop_train_fn_01cf9492,TERMINATED,,0.5,-16.1927,1616.09,10000,20
coop_train_fn_01d00e2c,RUNNING,128.3.28.231:7245,0.9,-17.184,1620.04,10000,20


100%|██████████| 20/20 [45:43<00:00, 137.17s/it]


Trial name,status,loc,gamma,reward,total time (s),ts,iter
coop_train_fn_01ce5c76,TERMINATED,,0,-33.3417,1616.1,10000,20
coop_train_fn_01cf1a8a,TERMINATED,,0.3,-19.0377,1623.6,10000,20
coop_train_fn_01cf9492,TERMINATED,,0.5,-16.1927,1616.09,10000,20
coop_train_fn_01d00e2c,TERMINATED,,0.9,-17.184,1620.04,10000,20


2020-03-02 14:54:14,656	INFO tune.py:352 -- Returning an analysis object by default. You can call `analysis.trials` to retrieve a list of trials. This message will be removed in future versions of Tune.
